In [2]:
from openai import OpenAI
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
import random
from tqdm import tqdm
import re
import pandas as pd
import langsmith as ls
import textwrap
from typing import TypedDict, Optional
from openai import OpenAI
from pydantic import BaseModel
import traceback
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
from langsmith import traceable, trace
from langsmith import Client, traceable, evaluate
import sys
import os
# Thêm đường dẫn project_root vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("dataset_to_langsmith.ipynb"), '..')))
from In_Context_Learning.preprocess_data import prepare_qa_input_with_answer_filter,standardize_item

In [3]:
file_path = 'test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    gsm8k = [json.loads(line) for line in f]

In [4]:
folder_path = ""
filename="tatqa_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    raw_data_tatqa = json.load(f)
tatqa_pre = prepare_qa_input_with_answer_filter(raw_data_tatqa)

In [5]:
folder_path = ""
filename="tabmwp_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    tabmwp_pre = json.load(f)

In [6]:
tatqa = []
for item in tatqa_pre:
    tatqa.extend(standardize_item(item, "tatqa"))
tatqa

[{'question': 'How much is the 2019 rate of inflation?',
  'answer': '2.9',
  'context': '|  | 2019 % | 2018 % | 2017 % |\n| --- | --- | --- | --- |\n| Weighted average actuarial assumptions used at 31 March1: |  |  |  |\n| Rate of inflation2 | 2.9 | 2.9 | 3.0 |\n| Rate of increase in salaries | 2.7 | 2.7 | 2.6 |\n| Discount rate | 2.3 | 2.5 | 2.6 |\n'},
 {'question': 'How much is the 2018 rate of inflation?',
  'answer': '2.9',
  'context': '|  | 2019 % | 2018 % | 2017 % |\n| --- | --- | --- | --- |\n| Weighted average actuarial assumptions used at 31 March1: |  |  |  |\n| Rate of inflation2 | 2.9 | 2.9 | 3.0 |\n| Rate of increase in salaries | 2.7 | 2.7 | 2.6 |\n| Discount rate | 2.3 | 2.5 | 2.6 |\n'},
 {'question': 'What is the 2019 average rate of inflation?',
  'answer': '2.9',
  'context': '|  | 2019 % | 2018 % | 2017 % |\n| --- | --- | --- | --- |\n| Weighted average actuarial assumptions used at 31 March1: |  |  |  |\n| Rate of inflation2 | 2.9 | 2.9 | 3.0 |\n| Rate of increase

In [7]:
tabmwp = []
for item in tabmwp_pre:
    tabmwp.extend(standardize_item(item, "tabmwp"))
tabmwp

[{'question': "The members of the local garden club tallied the number of plants in each person's garden. How many gardens have at least 47 plants?",
  'answer': '13',
  'context': 'Plants per garden\n\n| Stem | Leaf  |\n| --- | --- |\n| 3 | 3, 3, 3, 5, 5 |\n| 4 | 6 |\n| 5 | 4, 5, 7, 8 |\n| 6 | 7, 8 |\n| 7 | 2, 3, 7, 9 |\n| 8 | 6, 8, 9 |\n'},
 {'question': 'The Burlington Symphony categorizes its donors as gold, silver, or bronze depending on the amount donated. What fraction of donors are at the bronze level? Simplify your answer.',
  'answer': '58/141',
  'context': 'Donor levels\n\n| Donation level | Number of donors |\n| --- | --- |\n| Gold | 15 |\n| Silver | 68 |\n| Bronze | 58 |\n'},
 {'question': "Employees at Eve's Movies tracked the number of movies that customers rented last month. How many customers are there in all?",
  'answer': '44',
  'context': 'Renting movies last month\n\n| Number of times | Frequency |\n| --- | --- |\n| 0 | 1 |\n| 1 | 18 |\n| 2 | 12 |\n| 3 | 13 |\n| 

In [8]:
load_dotenv()

True

In [9]:
client = Client()

In [10]:
sample_size = 200
random.seed(42)  
test_samples = random.sample(list(tatqa), sample_size)

dataset_id = "d44eb0f3-ba86-4a8c-8486-5364a905e9db"

inputs = [{"question": item["question"], "context": item["context"]} for item in test_samples]
outputs = [{"answer": item["answer"]} for item in test_samples]

client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset_id,
)


{'example_ids': ['be535820-d4fc-4630-a850-ab35b7516492',
  'ecdf3c08-27e9-40ee-92c7-3500c737e248',
  '7a444c7a-b9b1-4818-8fcc-84210fc29a10',
  'ed964629-d907-45f4-bcf8-d9390244941f',
  '332eb283-2b6b-43cc-b168-cd13c016d080',
  '2899e570-00ff-4fae-ac86-32ed27870b8b',
  '0c17fd81-476b-4938-b7df-6becdd013df9',
  'a1a00f44-513f-4d42-9f49-41494d238f08',
  '955f5588-1335-4134-a816-803e3a04ecf7',
  '37fa6f11-39cd-42f6-a197-dc912256db33',
  '8b18d3fb-1810-42ae-9b93-3d97767f84b3',
  '440b6c70-db7d-4575-a9ad-ac42a0f237af',
  'a0baac54-2d61-4356-8e10-257c5646c1a5',
  'ca0cda41-e5de-463f-be81-46387d752099',
  'a099747f-b07c-4efd-92ed-02ddf6e58873',
  '1f282ae4-4f15-4e46-9d9e-6f1ebd26ae1e',
  'd5032efb-3adc-4a12-843e-c250bf30a009',
  'e7a6d08d-06eb-4174-9020-5d934160b812',
  '2264b74d-09df-4a2d-a483-45ee76ad1573',
  'c655f50f-5999-4cf3-aeaf-f18744397925',
  '41e6cbdf-12ac-4465-94ca-50f3a1546178',
  'a9c62aa6-c2fd-4d6e-8436-019f3087897b',
  '751315e6-4db6-43b6-8513-8822f1541fd3',
  '1e0ff912-8ab7-4b

In [11]:
sample_size = 200
random.seed(42)  
test_samples = random.sample(list(tabmwp), sample_size)

dataset_id = "eb4dd623-5fda-46f4-8535-aa2f9d48f874"

inputs = [{"question": item["question"], "context": item["context"]} for item in test_samples]
outputs = [{"answer": item["answer"]} for item in test_samples]

client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset_id,
)


{'example_ids': ['5082258d-6ba1-4928-96bc-38e4fe35527d',
  '1aa660ed-ca6b-4117-bed7-da62e59cb7b7',
  '7105ed4c-2188-463d-9ec7-128d6618ef9d',
  '2b378373-14b9-486d-af79-446007b5a5f3',
  '43f3e3ad-8c3f-46da-a66d-6e010f92ab9c',
  '1f2e72e7-4062-418e-b745-829cfc18a4e2',
  'cc9bf997-9d38-4f30-8a1c-69ebe39f85fe',
  '104f8dac-f2dd-4bd9-97c8-1babc37da9db',
  '45c9fe2d-8120-4acc-ba8a-c1901a3e603c',
  'e52a1fcd-e58f-4848-bc41-798d3c6ffbf9',
  '0536acf5-d205-4d21-9e9a-1c964fa316c9',
  '7fe1a03d-6e5e-4754-8659-e34313f977f2',
  '4a16bf1e-b8e6-41c3-8ab2-bb51f5bdd5f2',
  'b1d56791-b5ff-4087-ae59-77bb554bd51b',
  '42aa89ba-cd22-447e-b34d-1b52a466b194',
  '1dddf873-185a-4118-aab0-baf044183a28',
  'c9032817-b8d8-4e84-b411-893defb2747e',
  'd135dc81-db30-4e98-adf1-c0c74de73be3',
  '57a0e257-5a22-4151-bb79-726e64f9f990',
  '4b2fcbdd-752c-4f3e-a5fe-dee1ca05fd5e',
  '9cda9e81-c30b-4864-961b-77ee98527449',
  'cefaf876-f529-4585-ba41-3b86a0a51f65',
  'e78bb95b-94e5-4275-a22b-52b674b77896',
  'f08ffdbe-2320-44

In [12]:
sample_size = 200
random.seed(42)  
test_samples = random.sample(list(gsm8k), sample_size)

dataset_id = "ef71894e-207f-4f3d-9852-38533e36fbe5"

inputs = [{"question": item["question"]} for item in test_samples]
outputs = [{"answer": item["answer"]} for item in test_samples]

client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset_id,
)


{'example_ids': ['27a51ddd-5e3d-4d38-b28b-9405ce74843b',
  'ba178d79-0bb8-438c-9f26-0827de9f4362',
  '32acb3e5-32cd-4f2f-a047-6a680264ed23',
  '312a5094-cc15-4d52-be80-18e5e77674dc',
  'd0a9388d-8fc7-4e7d-ad28-bda194f221aa',
  'f89071b8-7de9-4e02-856a-a4af1407316c',
  'bbf93ad8-4c51-44e1-9caf-394c46e5f16f',
  'fbf39a0c-e7c3-4169-980e-49e367646326',
  '3cec929a-f001-41b5-9257-91c607433f71',
  '4af39acd-95e5-414b-b7d3-75b913167695',
  '0b5e4d3f-6aac-4d9d-9714-92daf299f2d2',
  '36cfdf04-4133-49df-916e-6a517fa022dc',
  '3f85919b-26d3-4a1d-a946-8cdc0abde71e',
  '857882a1-03fd-4f99-8e00-3057daafd4fe',
  'c8d69f6d-d138-4965-a948-abc9faa5d291',
  '186c9265-059d-408c-bb38-c9833d1c4dc7',
  '72db40e1-9875-4252-a5ea-8d6c9a2d459f',
  'f56d5357-bbc5-46b8-a682-221731fc0824',
  '6e46a157-0c7e-4c36-8c4c-f9ed8cb25817',
  'cac11847-4f93-44ae-b3de-972905af21aa',
  '71732b2a-d8e4-4df7-a096-cdd4bc1980fa',
  'b88810aa-1789-4293-8673-7119e75d2ae3',
  '06b34e2a-723d-42d0-aaa9-57e39e5652d3',
  '7b27dac1-f77c-42